In [374]:
import re
from collections import defaultdict
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from konlpy.tag import Okt

In [3]:
eip_pb = pd.read_csv('../Data/EIP_Problem_All.csv', encoding='euc-kr')
print(eip_pb.columns)
eip_pb.head()

Index(['PB_IDX', 'LI_IDX', 'EX_IDX', 'PB_NUM', 'PB_QUES', 'PB_DETAIL',
       'PB_CHOI1', 'PB_CHOI2', 'PB_CHOI3', 'PB_CHOI4', 'PB_ANS', 'PB_SOLU',
       'PB_CR', 'TOPIC_IDX'],
      dtype='object')


,PB_IDX,LI_IDX,EX_IDX,PB_NUM,PB_QUES,PB_DETAIL,PB_CHOI1,PB_CHOI2,PB_CHOI3,PB_CHOI4,PB_ANS,PB_SOLU,PB_CR,TOPIC_IDX
0,1,1,1,1,검토회의 전에 요구사항 명세서를 미리 배포하여 사전 검토한 후 짧은 검토 회의를 통...,NaN,빌드 검증,동료 검토,워크 스루,개발자 검토,3,이 문제는 요구사항 검토 방법 중에서 검토회의 *전*에 미리 자료를 배포하고 사전 ...,82,2
1,2,1,1,2,코드 설계에서 일정한 일련번호를 부여하는 방식의 코드는?,NaN,연상 코드,블록 코드,순차 코드,표의 숫자 코드,3,"코드 설계에서 일정한 일련번호를 부여하는 방식은 순차 코드입니다. 예를 들어, 상...",85,6
2,3,1,1,3,객체지향 프로그램에서 데이터를 추상화하는 단위는?,NaN,메소드,클래스,상속성,메시지,2,객체지향 프로그래밍(OOP)에서 데이터를 추상화하는 기본 단위는 클래스입니다. 클...,83,7
3,4,1,1,4,데이터 흐름도(DFD)의 구성요소에 포함되지 않는 것은?,NaN,process,data flow,data store,data dictionary,4,데이터 흐름도(DFD)는 시스템의 데이터 흐름을 그림으로 나타내는 기법입니다. 마...,83,3
4,5,1,1,5,소프트웨어 설계시 구축된 플랫폼의 성능특성 분석에 사용되는 측정 항목이 아닌 것은?,NaN,응답시간(Response Time),가용성(Availability),사용률(Utilization),서버 튜닝(Server Tuning),4,"소프트웨어 설계 시 구축된 플랫폼의 성능 특성을 분석하려면, 시스템의 반응 속도, ...",90,1


In [4]:
image_to_text_df = pd.read_csv('../Data/EIP_Image_to_Text.csv')
pb_detail_text = image_to_text_df['PB_DETAIL_TEXT'].tolist()
pb_choi1_text = image_to_text_df['PB_CHOI1_TEXT'].tolist()
pb_choi2_text = image_to_text_df['PB_CHOI2_TEXT'].tolist()
pb_choi3_text = image_to_text_df['PB_CHOI3_TEXT'].tolist()
pb_choi4_text = image_to_text_df['PB_CHOI4_TEXT'].tolist()

In [201]:
topic_descriptions = {
    1: "플랫폼 구성, 시스템 성능, 병목 지점, 운영체제 종류, 네트워크 연결, DBMS 종류, 응답 시간, 트랜잭션 처리량, TPS, 시스템 아키텍처, 플랫폼 기능, 성능 특성, 운영체제, 네트워크, DBMS, 비즈니스융합분석",
    2: "워크스루, 인스펙션, 프로토타이핑, 요구사항 도출, UML, Agile, 요구공학, Use Case, 이해관계자 분석, 요구 명세서, 요구 검증, 요구관리 도구, 요구 분석 기법, UML, 애자일",
    3: "분석 모델, 클래스 다이어그램, 시퀀스 다이어그램, 요구 추적성, CASE 도구, 분석 자동화, 유스케이스, 모델 검증, 모델링 기법, 분석자동화 도구, 요구사항 관리 도구, UML 다이어그램",
    4: "UI 구성요소, UX, UI 흐름도, 스토리보드, 인터페이스 명세, UI 표준화, UI 요구사항 문서, UI 표준, UI 지침, 스토리보드",
    5: "감성공학, UI 설계 도구를 포함한 UI 설계는 직관적이고 사용자 친화적인 인터페이스를 설계하는 주제입니다. 사용자 감정을 고려한 디자인과 도구를 활용해 효율적인 UI를 구현합니다. 주요 키워드: UI 설계, 감성공학, UI 툴킷, 인터페이스 레이아웃, 사용자 편의성, 피드백 요소.",
    6: "설계 모델링, 소프트웨어 아키텍처, 재사용, 모듈화, 결합도, 응집도를 포함한 공통 모듈 설계는 소프트웨어의 재사용 가능한 구성 요소를 설계하는 주제입니다. 모듈 간 독립성을 높이고 시스템 구조를 최적화합니다. 주요 키워드: 공통모듈, 아키텍처 설계, 모듈화, 응집도, 결합도, 컴포넌트, 재사용 설계, 계층 구조.",
    7: "주요 키워드: 객체지향 설계, 클래스 설계, 캡슐화, 상속, 다형성, 디자인 패턴, SOLID 원칙, 객체지향 프로그래밍(OOP), 디자인 패턴, 클래스, 상속, 다형성, 캡슐화",
    8: "내외부 인터페이스 요구사항, 요구공학을 포함한 인터페이스 요구사항 확인은 시스템 간 데이터 교환 요구사항을 정의하는 주제입니다. 인터페이스의 기능과 표준을 명확히 설정해 상호운용성을 확보합니다. 주요 키워드: 인터페이스 요구사항, 데이터 포맷 정의, API 명세, 인터페이스 규약, 표준 인터페이스.",
    9: "시스템 아키텍처, 인터페이스 시스템을 포함한 인터페이스 대상 식별은 시스템 간 연결 대상을 식별하는 주제입니다. 인터페이스의 구조와 역할을 분석해 통합 설계를 지원합니다. 주요 키워드: 인터페이스 식별, 시스템 아키텍처 분석, 인터페이스 다이어그램, 통합 포인트.",
    10: "내외부 송수신, 데이터 명세화, 오류 처리 방안 명세화, 미들웨어 솔루션을 포함한 인터페이스 상세 설계는 인터페이스의 세부 구현을 설계하는 주제입니다. 데이터 흐름과 오류 대응 전략을 구체화합니다. 주요 키워드: 인터페이스 상세 설계, 오류 처리, 미들웨어, 송수신 구조, 데이터 흐름도, 통신 프로토콜.",
    11: "스택, 큐, 리스트, 트리, 그래프를 포함한 자료구조는 데이터 저장과 처리의 효율성을 높이는 주제입니다. 알고리즘과 구조를 활용해 검색, 정렬, 순회 등의 작업을 최적화합니다. 주요 키워드: 선형 구조, 비선형 구조, 스택, 큐, 트리, 그래프, 순회, 삽입, 삭제, 연결 리스트.",
    12: "프로시저, 프로그램 디버깅, 단위테스트 도구를 포함한 데이터 조작 프로시저 작성은 데이터 처리 로직을 구현하는 주제입니다. 코드 안정성을 위해 디버깅과 테스트를 수행합니다. 주요 키워드: SQL 프로시저, PL/SQL, 디버깅, 단위 테스트, 에러 핸들링, 스크립트 작성.",
    13: "쿼리 성능 측정, 소스코드 인스펙션을 포함한 데이터 조작 프로시저 최적화는 데이터 처리 성능을 개선하는 주제입니다. 쿼리 튜닝과 코드 검토를 통해 효율성을 높입니다. 주요 키워드: 쿼리 튜닝, 인덱스 활용, 성능 분석, 실행 계획, 코드 인스펙션.",
    14: "단위모듈 구현, 단위모듈 테스트를 포함한 모듈 구현은 소프트웨어의 개별 구성 요소를 개발하고 검증하는 주제입니다. 각 모듈의 독립성과 기능을 보장합니다. 주요 키워드: 모듈 개발, 하향식 설계, 단위 테스트, 테스트 케이스, 기능 검증.",
    15: "IDE 도구, 협업도구, 형상관리 도구를 포함한 통합 구현 관리는 소프트웨어 개발 환경을 관리하는 주제입니다. 팀 협업과 코드 버전 관리를 통해 개발 효율성을 높입니다. 주요 키워드: Git, GitHub, 협업 툴, CI/CD, 형상관리, 버전관리.",
    16: "애플리케이션 패키징, 배포 도구, 모니터링 도구, DRM을 포함한 제품소프트웨어 패키징은 소프트웨어를 배포 가능한 형태로 만드는 주제입니다. 안정적인 배포와 저작권 보호를 목표로 합니다. 주요 키워드: 소프트웨어 배포, 패키징, 설치 파일, DRM, drm, 배포 자동화.",
    17: "제품소프트웨어 매뉴얼 작성, 국제 표준 제품 품질 특성을 포함한 매뉴얼 작성은 사용자와 개발자를 위한 문서를 작성하는 주제입니다. ISO/IEC 품질 표준을 준수해 명확성을 확보합니다. 주요 키워드: 사용자 설명서, 기술 문서, 품질 표준, ISO/IEC, 제품 가이드.",
    18: "소프트웨어 버전관리 도구, 빌드 자동화 도구를 포함한 제품소프트웨어 버전관리는 소프트웨어 변경 사항을 체계적으로 관리하는 주제입니다. Git, Jenkins 등을 활용해 안정성을 유지합니다. 주요 키워드: 버전 관리, Git, Jenkins, 자동 빌드, 배포 자동화.",
    19: "테스트 케이스, 테스트 레벨, 테스트 시나리오, 테스트 지식 체계를 포함한 애플리케이션 테스트케이스 설계는 소프트웨어 품질을 검증하는 주제입니다. 체계적인 테스트 계획을 수립합니다. 주요 키워드: 테스트 설계, 테스트 케이스, 테스트 시나리오, 테스트 커버리지.",
    20: "결함관리 도구, 테스트 자동화 도구, 통합 테스트를 포함한 애플리케이션 통합 테스트는 모듈 간 상호작용을 검증하는 주제입니다. 시스템 전체의 안정성을 보장합니다. 주요 키워드: 통합 테스트, 자동화 테스트, 결함 추적, 테스트 툴, 시스템 검증.",
    21: "알고리즘, 소스코드 품질분석 도구, 코드 최적화를 포함한 애플리케이션 성능 개선은 소프트웨어 실행 효율성을 높이는 주제입니다. 최적화 기법으로 성능 병목을 해결합니다. 주요 키워드: 알고리즘 최적화, 시간복잡도, 성능 분석, 코드 개선, 메모리 사용량.",
    22: "인터페이스 기능 확인, 데이터 표준 확인을 포함한 인터페이스 설계 확인은 인터페이스 설계의 적합성을 검증하는 주제입니다. 표준 준수와 기능 완성도를 평가합니다. 주요 키워드: 인터페이스 검증, 데이터 포맷, 설계 기준, 표준 준수, API 확인.",
    23: "인터페이스 보안, 소프트웨어 연계 테스트를 포함한 인터페이스 기능 구현은 인터페이스를 실제로 구현하는 주제입니다. 보안성과 연계성을 테스트해 안정성을 확보합니다. 주요 키워드: 인터페이스 구현, 보안 연계, 연계 테스트, 인증, 암호화, API 연동.",
    24: "설계 산출물, 인터페이스 명세서를 포함한 인터페이스 구현 검증은 인터페이스 구현 결과를 검증하는 주제입니다. 명세서와의 일치성을 확인해 품질을 보장합니다. 주요 키워드: 인터페이스 테스트, 구현 검증, 명세 일치, 결과 보고서, 시스템 연계 검토.",
    25: "트리거, 이벤트, 사용자 정의 함수, SQL 문법을 포함한 절차형 SQL 작성은 데이터베이스에서 동적 쿼리를 작성하는 주제입니다. 자동화된 데이터 처리를 구현합니다. 주요 키워드: 프로시저, 트리거, 이벤트, UDF, 제어 흐름, SQL 프로그래밍.",
    26: "DML, DCL, 윈도우 함수, 그룹 함수, 오류 처리를 포함한 응용 SQL 작성은 데이터 조작과 권한 관리를 위한 쿼리를 작성하는 주제입니다. 복잡한 데이터 분석을 지원합니다. 주요 키워드: SELECT, JOIN, 서브쿼리, 권한 부여, 윈도우 함수, 오류 처리.",
    27: "DDL, 관계형 데이터 모델, 트랜잭션, 테이블, 데이터 사전을 포함한 기본 SQL 작성은 데이터베이스 구조를 정의하는 주제입니다. 기본적인 테이블 생성과 관리 기술을 다룹니다. 주요 키워드: CREATE, ALTER, DROP, 테이블 설계, 트랜잭션 제어.",
    28: "뷰, 인덱스, 집합 연산자, 조인, 서브쿼리를 포함한 고급 SQL 작성은 복잡한 데이터 조회를 위한 쿼리를 작성하는 주제입니다. 성능과 가독성을 최적화합니다. 주요 키워드: 인덱스, 서브쿼리, 조인, 집합 연산자, WITH절, 복합 쿼리.",
    29: "관계 데이터 모델, 관계대수, 관계해석, 시스템카탈로그, 뷰를 포함한 관계데이터베이스 모델은 데이터베이스의 이론적 기반을 다루는 주제입니다. 데이터 구조와 연산을 정의합니다. 주요 키워드: 관계대수, 릴레이션, 도메인, 튜플, 카탈로그.",
    30: "데이터모델 개념, E-R 모델, 논리적 데이터모델링, 정규화, 품질검증을 포함한 데이터모델링 및 설계는 데이터베이스 논리 구조를 설계하는 주제입니다. 데이터 일관성과 효율성을 보장합니다. 주요 키워드: 정규화, 비정규화, 엔티티, 관계, 속성, E-R 모델.",
    31: "스토리지, 분산 데이터베이스, 이중화 구성, 암호화, 접근제어를 포함한 물리요소 조사 분석은 데이터베이스의 물리적 환경을 분석하는 주제입니다. 안정성과 보안성을 평가합니다. 주요 키워드: 스토리지, RAID, 이중화, 장애 대응, 접근 제어, 물리 보안, 분산 저장.",
    32: "파티셔닝, 클러스터링, 백업, 테이블 저장 사이징, 데이터 지역화를 포함한 데이터베이스 물리속성 설계는 물리적 저장 구조를 최적화하는 주제입니다. 성능과 안정성을 높입니다. 주요 키워드: 파티셔닝, 클러스터링, 백업 정책, 테이블 사이징, 지역화 전략.",
    33: "데이터베이스 무결성, 칼럼 속성, 키 종류, 반정규화, 정규화를 포함한 물리 데이터베이스 모델링은 데이터베이스의 물리적 구조를 설계하는 주제입니다. 데이터 일관성을 유지합니다. 주요 키워드: 기본 키, 외래 키, 무결성 제약조건, 정규화, 반정규화.",
    34: "물리데이터 모델 품질 기준, E-R 다이어그램, CRUD 분석, SQL 성능 튜닝을 포함한 물리데이터 모델 품질 검토는 데이터베이스 설계의 품질을 검증하는 주제입니다. 성능과 구조를 최적화합니다. 주요 키워드: 데이터 모델 검토, 튜닝 포인트, ERD, CRUD 매핑.",
    35: "초기데이터 구축, ETL, 파일 처리 기술을 포함한 데이터 전환 기술은 데이터를 새로운 시스템으로 전환하는 주제입니다. 데이터 무결성과 효율적인 마이그레이션을 보장합니다. 주요 키워드: 데이터 이전, ETL, 정제, 마이그레이션, 변환 룰.",
    36: "데이터 전환 수행 계획, 체크리스트, 데이터 검증을 포함한 데이터 전환 수행은 데이터 마이그레이션을 실행하는 주제입니다. 전환 과정의 정확성을 검증합니다. 주요 키워드: 전환 계획, 검증 절차, 테스트 전환, 체크리스트 기반 검토.",
    37: "데이터 품질 분석, 오류 데이터 측정을 포함한 데이터 정제는 데이터의 정확성과 완전성을 높이는 주제입니다. 데이터 오류를 식별하고 정정합니다. 주요 키워드: 데이터 클렌징, 오류 탐지, 정제 규칙, 품질 지표, 표준화.",
    38: "개발환경 구축, 서버 개발 프레임워크를 포함한 개발환경 구축은 소프트웨어 개발을 위한 인프라를 준비하는 주제입니다. 개발 효율성과 안정성을 지원합니다. 주요 키워드: IDE 설치, 개발 도구 세팅, 프레임워크 구성, 환경 설정.",
    39: "보안 취약성 식별, API를 포함한 서버 프로그램 구현은 서버 측 애플리케이션을 개발하는 주제입니다. 안전하고 효율적인 서버 로직을 구현합니다. 주요 키워드: 서버 개발, REST API, 인증, 세션 처리, 취약점 식별.",
    40: "배치 프로그램을 포함한 배치 프로그램 구현은 대량 데이터 처리를 자동화하는 주제입니다. 주기적 작업을 효율적으로 수행합니다. 주요 키워드: 배치 처리, 예약 작업, Cron, 대용량 처리, 로그 관리.",
    41: "데이터 타입, 변수, 연산자를 포함한 기본 문법 활용은 프로그래밍 언어의 기본 요소를 사용하는 주제입니다. 코드 작성의 기초를 다룹니다. 주요 키워드: 변수 선언, 자료형, 연산자, 조건문, 반복문, 함수.",
    42: "절차적, 객체지향, 스크립트, 선언형 언어를 포함한 언어특성 활용은 다양한 프로그래밍 언어의 특성을 활용하는 주제입니다. 언어별 최적의 코딩 방식을 적용합니다. 주요 키워드: 언어 유형, 객체지향, 함수형, 스크립트 언어, 특성 비교.",
    43: "라이브러리, 데이터 입출력, 예외 처리, 프로토타입을 포함한 라이브러리 활용은 소프트웨어 기능을 확장하는 주제입니다. 재사용 가능한 코드를 효율적으로 사용합니다. 주요 키워드: 라이브러리 호출, API 사용, 예외처리, 입출력 처리.",
    44: "운영체제 종류, 메모리 관리, 프로세스 스케쥴링, 환경변수, shell script를 포함한 운영체제 기초 활용은 시스템 자원을 관리하는 주제입니다. OS의 기본 기능을 활용합니다. 주요 키워드: 프로세스 관리, 메모리 할당, 스케줄링, Shell 명령어.",
    45: "인터넷 구성, 네트워크 7계층, IP, TCP/UDP를 포함한 네트워크 기초 활용은 네트워크 구조와 프로토콜을 이해하는 주제입니다. 데이터 통신의 기초를 다룹니다. 주요 키워드: OSI 7계층, TCP/IP, 포트번호, 라우팅, DNS.",
    46: "웹서버, DB서버, 패키지를 포함한 기본 개발환경 구축은 소프트웨어 개발을 위한 서버 환경을 구성하는 주제입니다. 개발 준비를 체계화합니다. 주요 키워드: 웹서버 설정, DB 연결, 아파치, 톰캣, 서버 패키지.",
    47: "소프트웨어 생명주기 모델, 개발 방법론, 요구공학 방법론, 비용산정 모델을 포함한 소프트웨어개발 방법론 선정은 프로젝트에 적합한 개발 방식을 선택하는 주제입니다. 효율적인 개발 프로세스를 설계합니다. 주요 키워드: 폭포수 모델, 애자일, 스크럼, COCOMO, 요구 분석.",
    48: "소프트웨어 개발 표준, 테일러링 기준, 개발 프레임워크를 포함한 소프트웨어개발 방법론 테일러링은 개발 프로세스를 프로젝트에 맞게 조정하는 주제입니다. 유연한 개발 방식을 적용합니다. 주요 키워드: 프로세스 테일러링, 표준 프레임워크, 커스터마이징, 적용 사례.",
    49: "IT 신기술, 네트워크 장비(라우터, 백본 스위치)를 포함한 네트워크 구축 관리는 네트워크 인프라를 설계하고 관리하는 주제입니다. 최신 기술을 활용해 안정성을 높입니다. 주요 키워드: 라우터, 스위치, 네트워크 토폴로지, VLAN, 망 구성.",
    50: "IT 신기술, SW 개발 트렌드, 보안 정책을 포함한 SW 구축 관리는 소프트웨어 개발 프로젝트를 관리하는 주제입니다. 최신 트렌드와 보안을 반영합니다. 주요 키워드: 개발 정책, 정보보안, 프로젝트 관리, 보안 가이드, 신기술 적용.",
    51: "IT 신기술, 서버장비 운영(Secure-OS, NAS, DAS, SAN, 고가용성)을 포함한 HW 구축 관리는 하드웨어 인프라를 관리하는 주제입니다. 시스템 안정성과 가용성을 보장합니다. 주요 키워드: 서버 장비, NAS, SAN, 고가용성 구성, 하드웨어 관리.",
    52: "IT 신기술, 데이터베이스 관리 기능, 표준화를 포함한 DB 구축 관리는 데이터베이스 시스템을 설계하고 관리하는 주제입니다. 데이터 관리의 효율성을 높입니다. 주요 키워드: DBMS 운영, 성능 관리, 표준화 정책, 데이터 보안, DB 설계.",
    53: "Secure SDLC, 입력데이터 검증, 인증, 접근제어, 세션통제를 포함한 SW개발 보안 설계는 소프트웨어 개발에서 보안을 강화하는 주제입니다. 보안 취약점을 최소화합니다. 주요 키워드: 보안 설계, 입력 검증, 세션 관리, 접근 제어, SDLC.",
    54: "암호 알고리즘, 코드오류, 캡슐화, API 오용을 포함한 SW개발 보안 구현은 보안 기능을 실제로 구현하는 주제입니다. 안전한 코드를 작성합니다. 주요 키워드: 암호화, 암호 알고리즘, 캡슐화, 안전 코딩, 보안 라이브러리.",
    55: "서비스 공격 유형, 서버 인증, 접근통제, 보안 아키텍처를 포함한 시스템 보안 설계는 시스템 전체의 보안을 설계하는 주제입니다. 외부 위협에 대응합니다. 주요 키워드: 보안 정책, 접근 통제, 인증 메커니즘, 공격 대응, 방화벽.",
    56: "로그 분석, 보안 솔루션, 취약점 분석을 포함한 시스템 보안 구현은 보안 시스템을 운영하고 점검하는 주제입니다. 시스템의 취약점을 식별하고 대응합니다. 주요 키워드: 보안 솔루션, 로그 모니터링, 취약점 진단, 위협 탐지, 침입 대응.",
}


In [371]:
pb_choi_texts = [pb_choi1_text, pb_choi2_text, pb_choi3_text, pb_choi4_text]

def format_choice(n, val, index):
        val = str(val)
        if val.startswith('http://') or val.startswith('https://'):
            return f"{n}. {pb_choi_texts[n-1][index]}"
        else:
            return f"{n}. {val}"

In [372]:
# 텍스트 생성
texts = []
for index in range(len(eip_pb)):
    text = f"{eip_pb['PB_QUES'][index]}"
    
    # 사진(경로)가 있을 때 PB_DETAIL 추가
    if pd.notna(eip_pb['PB_DETAIL'][index]):
        text += f"""
            {pb_detail_text[index]}
        """
    
    # 정답 선지 추가
    ans = eip_pb['PB_ANS'][index]
    ans_val = eip_pb[f'PB_CHOI{ans}'][index]
    text += f"\n{format_choice(ans, ans_val, index)}"
    
    # 해설 추가
    text += f"\n{eip_pb['PB_SOLU'][index]}"
    
    texts.append(text)

In [ ]:
# 텍스트를 데이터프레임에 추가
eip_pb['combined_text'] = texts

stopwords = [
    '만드는', '있는', '것은', '있도록', '있습니다', '것이', '옳지', '위한', 
    '있어야', '됩니다', '서로', '간의', '대한', '등을', '하고', '없이', 
    '다음', '먼저', '가장', '매우', '하나의', '모든', '전에'
]

# 주제별 키워드 분석
vectorizer = TfidfVectorizer(max_features=2000, ngram_range=(1, 2), min_df=1, max_df=0.9, stop_words=stopwords, lowercase=False)  # 한국어 불용어 리스트 추가 가능
X = vectorizer.fit_transform(eip_pb['combined_text'])

# 주제별 키워드 추출
topic_keywords = {}
for topic in eip_pb['TOPIC_IDX'].unique():
    topic_mask = eip_pb['TOPIC_IDX'] == topic
    topic_vector = X[topic_mask].mean(axis=0)
    feature_names = vectorizer.get_feature_names_out()
    top_feats = list(dict.fromkeys([feature_names[i] for i in np.argsort(topic_vector.A)[0, -20:]]))
    topic_keywords[topic] = top_feats[-20:]

okt = Okt()
keyword_texts = [
    ' '.join(dict.fromkeys([  # ✅ 중복 제거 + 순서 유지
        w for w, pos in okt.pos(' '.join(topic_keywords[topic]))
        if pos in ['Noun', 'Alpha', 'Adjective'] and w not in stopwords
    ]))
    for topic in sorted(topic_keywords.keys())
]

In [364]:
def print_tfidf_vector(tfidf_matrix, vectorizer, index, label='Vector'):
    feature_names = vectorizer.get_feature_names_out()
    vector = tfidf_matrix[index].toarray()[0]  # 희소 행렬 -> 밀집 배열
    nonzero_indices = vector.nonzero()[0]
    sorted_indices = nonzero_indices[np.argsort(vector[nonzero_indices])[::-1]]
    
    print(f"\n🔍 {label} (index: {index})")
    for i in sorted_indices:
        print(f"{feature_names[i]:<20} : {vector[i]:.4f}")

# 각 문서별로 실제 등장한 단어들 (non-zero TF-IDF) 추출
def extract_nonzero_terms(tfidf_vector):
    indices = tfidf_vector.nonzero()[1]
    return set([feature_names[i] for i in indices])
        
from sklearn.metrics.pairwise import cosine_similarity

# 한국어 형태소 분석
okt = Okt()
keyword_texts = [' '.join([word for word in okt.nouns(' '.join(topic_keywords[topic])) if word not in stopwords]) 
                 for topic in sorted(topic_keywords.keys())]
description_texts = [' '.join([word for word in okt.nouns(topic_descriptions[topic].split(' 등을 포함한 ')[0]) if word not in stopwords]) 
                    for topic in sorted(topic_keywords.keys())]

# TF-IDF 벡터화
X_keywords = vectorizer.fit_transform(keyword_texts)
X_descriptions = vectorizer.transform(description_texts)

In [ ]:
i = 6
print_tfidf_vector(X_keywords, vectorizer, i, label='X_keywords')
print_tfidf_vector(X_descriptions, vectorizer, i, label='X_descriptions')


🔍 X_keywords (index: 6)
객체                   : 0.5485
클래스                  : 0.3962
패턴                   : 0.3291
객체 클래스               : 0.2194
디자인                  : 0.2194
행위                   : 0.2194
행위 클래스               : 0.2194
패턴 객체                : 0.2194
클래스 클린               : 0.1097
객체 객체지향              : 0.1097
패턴 디자인               : 0.1097
객체 패턴                : 0.1097
객체 행위                : 0.1097
객체지향                 : 0.1097
다른 행위                : 0.1097
클래스 패턴               : 0.1097
디자인 객체               : 0.1097
디자인 패턴               : 0.1097
클린 객체                : 0.1097
클린                   : 0.1097
클래스 다른               : 0.1097
클래스 디자인              : 0.1097
상위 객체                : 0.1097
상위                   : 0.0915
다른                   : 0.0649

🔍 X_descriptions (index: 6)
패턴                   : 0.4054
디자인 패턴               : 0.4054
디자인                  : 0.4054
객체지향                 : 0.4054
클래스                  : 0.3661
설계                   : 0.3661
워드              